In [ ]:
import numpy as np
import torch 
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import os
import joblib
import glob
from skimage import io
import random
import matplotlib.pyplot as plt

In [ ]:
class DenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.densenet = models.densenet201(pretrained=True)
        self.fc = nn.Linear(1000, 6)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.densenet(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, mode, transform, val_size):
        super().__init__()
        self.transform = transform
        self.mode = mode
        path1 = '/kaggle/input/intel-image-classification/seg_train/seg_train/**/*.jpg'
        path2 = '/kaggle/input/intel-image-classification/seg_test/seg_test/**/*.jpg'
        all_paths = glob.glob(path1, recursive=True) + glob.glob(path2, recursive=True)
        random.Random(0).shuffle(all_paths)
        if mode == 'train':
            self.images_paths = all_paths[0: int(len(all_paths) * (1 - val_size))]
        elif mode == 'val':
            self.images_paths = all_paths[int(len(all_paths) * (1 - val_size)): ]
        self.labels_to_int = {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}

    def __getitem__(self, index):
        img = io.imread(self.images_paths[index])
        img = self.transform(img)
        label = self.images_paths[index].split(os.sep)[-2]
        label = self.labels_to_int[label]
        return img.float(), label

    def __len__(self):
        return len(self.images_paths)

In [ ]:
loss_func = nn.CrossEntropyLoss()
batch_size = 64
learning_rate = 0.001
num_epochs = 7
val_size = 0.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = DenseNet()
model = model.to(device)
train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((150, 150)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.GaussianBlur(5),
                                      transforms.RandomAdjustSharpness(2),
                                      transforms.RandomAutocontrast(),
                                      transforms.ColorJitter(brightness=0.5, hue=0.3)])
val_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((150, 150))])
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
train_dataset = MyDataset('train', train_transform, val_size)
val_dataset = MyDataset('val', val_transform, val_size)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
for epoch in range(1, num_epochs + 1):
    print('epoch:', epoch)
    model.train()
    for samples, targets in train_loader:
        samples = samples.to(device)
        targets = targets.to(device)
        preds = model(samples)
        loss = loss_func(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
    
    model.eval() 
    with torch.no_grad():
        for mode, loader in zip(['train', 'val'], [train_loader, val_loader]):
            epoch_loss, currect, num_samples = 0, 0, 0
            for samples, targets in loader:
                samples = samples.to(device)
                targets = targets.to(device)
                preds = model(samples)
                loss = loss_func(preds, targets)
                epoch_loss += loss.item() * targets.shape[0] 
                currect += (torch.argmax(preds, axis=1) == targets).sum()
                num_samples += targets.shape[0]

            epoch_loss = epoch_loss / num_samples
            accuracy = currect / num_samples
            print(mode, '- loss:', f'{epoch_loss:.2}', 'accuracy:', f'{accuracy:.2}')

In [ ]:
samples, targets = next(iter(val_loader))
samples = samples.to(device)
preds = model(samples)
preds = preds.detach().cpu().numpy()
targets = targets.detach().cpu().numpy()
samples = samples.permute(0, 2, 3, 1).detach().cpu().numpy()
preds = np.argmax(preds, axis=1)

int_to_labels = {0: 'buildings', 1: 'forest', 2: 'glacier', 3: 'mountain', 4: 'sea', 5: 'street'}

for i in range(10):
    fig, ax = plt.subplots(1)
    ax.set_title('ground truth - ' + int_to_labels[targets[i]])
    ax.set_xlabel('pred - ' + int_to_labels[preds[i]])
    ax.imshow(samples[i])
    
fig.show()    